PART I

Install packages

```bash
pip3 install sqlalchemy
pip install pandas
pip install psycopg2
```

In [1]:
import configparser 
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import hashlib
from psycopg2.errors import UniqueViolation

In [13]:
#Method for Get database connection
def get_connection(user, password, host, port, db_name):
     conn_string = f"host={host} dbname={db_name} user={user} password={password} port={port}"
     try:
          connection = psycopg2.connect(conn_string)
          connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
          return connection
     except Exception as e:
        print("An error occurred while creating the database connection:", e)

In [14]:
#Receives the connection, the table name and creates the table with the selected attributes
def create_table(cursor, table_name): 
    try:
        query = sql.SQL("CREATE TABLE IF NOT EXISTS {} ("
                            "id VARCHAR(200) PRIMARY KEY, "
                            "codigo_proyecto INTEGER, "
                            "fecha DATE, "
                            "cedula VARCHAR(50),"
                            "nombre_completo VARCHAR(150), "
                            "especialidad VARCHAR(50), "
                            "horas INTEGER"
                            ")").format(sql.Identifier(table_name))
        cursor.execute(query)
    except Exception as e:
        print("An error occurred while creating table:", e)


In [15]:
#Receives the cursor, table_name, df_project for insert values in the table created
def insert_data(cursor, table_name, df_project):
    for index, row in df_project.iterrows():
        try: 
            hash_object = hashlib.md5(f"{row['fecha']}{row['cedula']}".encode())
            id = hash_object.hexdigest()
            query = sql.SQL("INSERT INTO {} (id, codigo_proyecto, fecha, cedula, nombre_completo, especialidad, horas) VALUES (%s, %s, %s, %s, %s, %s, %s)").format(sql.Identifier(table_name))
            cursor.execute(query, (id, row['proyecto'], row['fecha'], row['cedula'],  row['nombre_completo'], row['descripcion'], row['horas']))
        except Exception as e:
            print(f"Error: violación de la restricción UNIQUE con el id {id} - {e}")

In [16]:
def main(): 
    #A good way to avoid changing the code later is to create a config file where the paths of the CSV to be read are stored, so if in the future this path changes, it is easier to change it in the config file than to go and review and change every part of the code where it was used.
    config = configparser.ConfigParser()
    config.read('config.ini')
    #Creation of variables for each CSV stored in the config file, specifically in Paths.
    PROJECTS_FILE_PATH = config['Paths']['projects_file']
    EMPLOYEES_FILE_PATH = config['Paths']['employees_file']
    HOURS_WORKED_FILE_PATH = config['Paths']['hours_worked_file']
    SPECIALITY_FILE_PATH = config['Paths']['specialty_file']

    #Reading CSV files, in this case the information contains accent marks, so you have to use the encoding, it is also necessary to specify the delimiter as a semicolon since that is the characteristics of the CSV
    df_projects = pd.read_csv(PROJECTS_FILE_PATH,encoding='latin-1',delimiter=';')
    df_employees = pd.read_csv(EMPLOYEES_FILE_PATH,encoding='latin-1',delimiter=';')
    df_hours_worked = pd.read_csv(HOURS_WORKED_FILE_PATH,encoding='latin-1',delimiter=';')
    df_speciality = pd.read_csv(SPECIALITY_FILE_PATH,encoding='latin-1',delimiter=';')

    #Combine for create full name
    df_hours_worked = df_hours_worked.groupby(['cedula', 'fecha', 'proyecto'], as_index=False).agg({'horas':'sum'})
    df_employees['nombre_completo'] = df_employees['nombre'] + ' ' + df_employees['apellidos']
    df_hours_worked['fecha'] = pd.to_datetime(df_hours_worked['fecha'], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')
    df_employees = df_employees.merge(df_speciality, left_on='especialidad', right_on='codigo')
    df_merged = df_hours_worked.merge(df_employees, left_on='cedula', right_on='cedula')

    # Obtain database connection
    connection = get_connection('builddb', 'resdb', 'localhost', '5432', 'buildingsa')
    cursor = connection.cursor()

    for _, row in df_projects.iterrows():
        table_name = f"project_{row['descripcion'].lower().replace(",", "").replace(" ", "_")}"
        create_table(cursor, table_name)
        print('table', table_name, ' created' )
        
        df_project = df_merged[df_merged['proyecto'] == row['codigo']]
     
        # Insert data 
        df_project = df_project[['proyecto', 'fecha', 'cedula', 'nombre_completo', 'descripcion', 'horas']]
        insert_data(cursor, table_name, df_project)
        print('Data in ', table_name, ' aggregated' )
    
    cursor.close()
    connection.close()

In [17]:
main()

table project_construcción_bodegas_wallmart_cartago  created
Data in  project_construcción_bodegas_wallmart_cartago  aggregated
table project_construcción_edificio_alpha_alajuela  created
Data in  project_construcción_edificio_alpha_alajuela  aggregated
table project_remodelación_mall_desamparados  created
Data in  project_remodelación_mall_desamparados  aggregated
table project_remodelación_museo_de_los_niños  created
Data in  project_remodelación_museo_de_los_niños  aggregated


In [21]:
connection = get_connection('builddb', 'resdb', 'localhost', '5432', 'buildingsa')
df_project_walmart = pd.read_sql_query('SELECT * FROM project_construcción_bodegas_wallmart_cartago', connection)
df_project_alpha = pd.read_sql_query('SELECT * FROM project_construcción_edificio_alpha_alajuela', connection)
df_project_mall = pd.read_sql_query('SELECT * FROM project_remodelación_mall_desamparados', connection)
df_project_museo = pd.read_sql_query('SELECT * FROM project_remodelación_museo_de_los_niños', connection)

C:\Users\adria\AppData\Local\Temp\ipykernel_8392\1842139154.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_project_walmart = pd.read_sql_query('SELECT * FROM project_construcción_bodegas_wallmart_cartago', connection)
C:\Users\adria\AppData\Local\Temp\ipykernel_8392\1842139154.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_project_alpha = pd.read_sql_query('SELECT * FROM project_construcción_edificio_alpha_alajuela', connection)
C:\Users\adria\AppData\Local\Temp\ipykernel_8392\1842139154.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Plea

In [23]:
df_project_walmart

,id,codigo_proyecto,fecha,cedula,nombre_completo,especialidad,horas
0,ac145e49a39f089d572bcfae76e83791,1,2023-05-10,1-1060-8032,GEOVANNY RAMIRO CABEZAS VELASCO,Mensajero,3
1,d3801f66c2718c1d0f61c25736fb466b,1,2023-08-16,1-1060-8032,GEOVANNY RAMIRO CABEZAS VELASCO,Mensajero,1
2,ff5dd8e75f01b03e4afa169cdf939e52,1,2023-01-11,1-1749-6635,JOSEPH HUMBERTO TRAVEZ VILLALBA,Electricista,5
3,e7aac8a7af138052a31c56e5ed46f0f0,1,2023-11-09,1-1749-6635,JOSEPH HUMBERTO TRAVEZ VILLALBA,Electricista,3
4,841eb2fa2b245849b5f6c3dd4d7f13c1,1,2023-07-22,1-2195-5917,CARMEN LUCILA JORDAN MEDINA,Ayudante Carpintero,6
...,...,...,...,...,...,...,...
129,13f09640f4d9c426d08eb958eecf3584,1,2023-05-07,7-5993-3255,FRANCISCO EDUARDO YÉPEZ CADENA,Albañil Nivel 1,6
130,31e309d55307bc4345afc1797c2402c6,1,2023-06-10,7-6713-4209,IVÁN FRANCISCO RIVADENEIRA TORRES,Albañil Nivel 1,3
131,e4004c608d989c24cc04d7110149d75f,1,2023-10-02,7-8525-4984,KARINA ELIZABETH NIETO SUAREZ,Electricista,3
132,24d9547131b5092ea869eafc4c10239b,1,2023-09-25,7-8525-4984,KARINA ELIZABETH NIETO SUAREZ,Electricista,4


In [24]:
df_project_alpha

,id,codigo_proyecto,fecha,cedula,nombre_completo,especialidad,horas
0,314a9a4d380133e0a3c668c4c9de3afb,2,2023-07-11,1-1060-8032,GEOVANNY RAMIRO CABEZAS VELASCO,Mensajero,5
1,6e10314807e28d70bb2a5785daf5005a,2,2023-07-28,1-1060-8032,GEOVANNY RAMIRO CABEZAS VELASCO,Mensajero,3
2,02de37f4e3b8655f511cc47af3abbbc4,2,2023-05-13,1-1749-6635,JOSEPH HUMBERTO TRAVEZ VILLALBA,Electricista,6
3,a06417259b165e86ea312dc444c898a7,2,2023-06-11,1-1896-7325,EDISON MAURICIO VILLAVICENCIO ORELLANA,Albañil Nivel 1,3
4,b5e4a50516e43c0a3ccdcf649f60350a,2,2023-06-07,1-2477-586,MARIO FRANCISCO JATIVA REYES,Electricista,6
...,...,...,...,...,...,...,...
123,3e4df0d99411bca1cfd2719407cae8a5,2,2023-09-10,7-6202-5766,DIANA ESPERANZA DELGADO RODRIGUEZ,Electricista,3
124,f28d087e0dddb6aae671da3955f50f02,2,2023-01-03,7-6713-4209,IVÁN FRANCISCO RIVADENEIRA TORRES,Albañil Nivel 1,3
125,1cce8dd3dabb634db43e42a819531910,2,2023-10-08,7-6713-4209,IVÁN FRANCISCO RIVADENEIRA TORRES,Albañil Nivel 1,2
126,45407df50939473240834e3567fe8aa9,2,2023-06-16,7-6756-8451,PABLO FERNANDO SALTOS GALARZA,Albañil Nivel 1,3


In [25]:
df_project_mall

,id,codigo_proyecto,fecha,cedula,nombre_completo,especialidad,horas
0,e2f06fa16aa5e09c7e8e3e3b91033490,3,2023-11-22,1-1060-8032,GEOVANNY RAMIRO CABEZAS VELASCO,Mensajero,3
1,71b9344a62bfe30d8ffc23e15d84c863,3,2023-12-06,1-2195-5917,CARMEN LUCILA JORDAN MEDINA,Ayudante Carpintero,4
2,1619d5f16a4054a0d42fea1207bf06ea,3,2023-04-27,1-2477-586,MARIO FRANCISCO JATIVA REYES,Electricista,5
3,995a404a154d2d0f3a8467064cff2dab,3,2023-09-22,1-2580-2329,HECTOR PATRICIO TAPIA RAMIREZ,Carpintero Nivel 1,6
4,c8223507451f89434b7ca1895c1ba119,3,2023-03-08,1-2580-2329,HECTOR PATRICIO TAPIA RAMIREZ,Carpintero Nivel 1,4
...,...,...,...,...,...,...,...
121,8479a880d628a6dba1cef4e24928c9b7,3,2023-09-14,7-6756-8451,PABLO FERNANDO SALTOS GALARZA,Albañil Nivel 1,6
122,11e3eb4f4ba9c41d5761e1b92f632443,3,2023-03-10,7-6772-6455,JESSICA ROSA MACIO CUEVA,Electricista,5
123,bd592a0468d914ffbaa0aae45efcd928,3,2023-05-11,7-6772-6455,JESSICA ROSA MACIO CUEVA,Electricista,3
124,541e3d7f90984c852001d71227028db2,3,2023-05-20,7-6772-6455,JESSICA ROSA MACIO CUEVA,Electricista,4


In [26]:
df_project_museo

,id,codigo_proyecto,fecha,cedula,nombre_completo,especialidad,horas
0,9860206d871f860dc54aed73e3418b9a,4,2023-03-22,1-1060-8032,GEOVANNY RAMIRO CABEZAS VELASCO,Mensajero,2
1,6a43c170c0b47695b1147a2cde7fe11e,4,2023-12-04,1-2477-586,MARIO FRANCISCO JATIVA REYES,Electricista,5
2,297712c7e8592a31d0897dd33b81d2ac,4,2023-09-13,1-2598-7426,LILIA EDITH GARCIA ANCHUNDIA,Mensajero,4
3,2b315c942e7b283c471f26e2c5d0912f,4,2023-06-12,1-5227-6229,KARINA ANNABELL CHAMORRO CEDEÑO,Mensajero,1
4,f234278a4c545f79829122b844de20de,4,2023-04-08,1-5227-6229,KARINA ANNABELL CHAMORRO CEDEÑO,Mensajero,5
...,...,...,...,...,...,...,...
105,dc5a46925072d787ec4c63661ea374df,4,2023-07-25,7-5993-3255,FRANCISCO EDUARDO YÉPEZ CADENA,Albañil Nivel 1,3
106,d1826fd385aa6b1d0a240ed34fe62b97,4,2023-06-23,7-6202-5766,DIANA ESPERANZA DELGADO RODRIGUEZ,Electricista,1
107,4acdf24e85eb99d5a5dfa17bff8b3812,4,2023-12-21,7-6756-8451,PABLO FERNANDO SALTOS GALARZA,Albañil Nivel 1,5
108,b736f9f878e5285459436841af2a9b5b,4,2023-05-05,7-6756-8451,PABLO FERNANDO SALTOS GALARZA,Albañil Nivel 1,4
